In [1]:
from selenium import webdriver
import time
from datetime import datetime, timedelta
from dateutil.parser import parse

def get_url_list():
    account = 'APDChina'
    start_date = 'May 2011'
    start_date_parsed = parse(start_date,default = datetime(2011, 5, 1))

    url_list = []
    date0 = start_date_parsed
    date1 = date0 + timedelta(weeks = 1)
    while date0 < datetime.now():
        url_list.append(
            'https://twitter.com/search?l=&q=from%3{}%20since%3A{}%20until%3A{}&src=typd'.format(
                account,date0.strftime('%Y-%m-%d'),date1.strftime('%Y-%m-%d')))
        date0 = (date0 + timedelta(weeks = 1))
        date1 = (date1 + timedelta(weeks = 1))
    return url_list

def scrape_one_tweet(tweet):
    innerlist = []
    innerlist.append(tweet.find_element_by_class_name('username').text)
    innerlist.append(tweet.find_element_by_class_name('time').text)
    innerlist.append(tweet.find_element_by_class_name('js-tweet-text-container').text)
    
    try:
        tweet.find_element_by_class_name('AdaptiveMedia-photoContainer')
        innerlist.append('Y')
    except:
        innerlist.append('N')
    try:
        tweet.find_element_by_class_name('AdaptiveMedia-video')
        innerlist.append('Y')
    except:
        innerlist.append('N')
        
    innerlist.append(tweet.find_element_by_css_selector(
        "[class='ProfileTweet-action ProfileTweet-action--reply']"
    ).text.replace('Reply','').strip())
    innerlist.append(tweet.find_element_by_css_selector(
        "[class='ProfileTweet-action ProfileTweet-action--retweet js-toggleState js-toggleRt']"
    ).text.replace('Retweet','').strip())
    innerlist.append(tweet.find_element_by_css_selector(
        "[class='ProfileTweet-action ProfileTweet-action--favorite js-toggleState']"
    ).text.replace('Like','').strip())
    
    innerlist.append(
        'https://twitter.com{}'.format(
            tweet.find_element_by_class_name('tweet').get_attribute('data-permalink-path')))
    return innerlist

def get_week_tweetsbox(browser):
    l0 = 0
    l1 = 1
    while l0 != l1:    #下拉到底端
        l0 = len(browser.find_elements_by_class_name('js-stream-item'))
        browser.execute_script("window.scrollTo(0,10000000);") #不管翻页多大也只是往下拉 每次最多多拿20个elments
        time.sleep(2)
        l1 = len(browser.find_elements_by_class_name('js-stream-item'))
    tweetsbox = browser.find_elements_by_xpath("//ol/li[@data-item-type='tweet']") #通过xpath查询
    return tweetsbox
    

In [ ]:
%%time
import csv
browser = webdriver.Chrome() 
url_list = get_url_list()
outerlist = []
for i in url_list:
    browser.get(i)
    time.sleep(1)
    try:
        tweetsbox = get_week_tweetsbox(browser)
    except:
        continue
    for m in tweetsbox:
        innerlist = scrape_one_tweet(m)
        
        with open('APDChina.csv','a') as f:
            mywriter=csv.writer(f)
            mywriter.writerow(innerlist)
            
        outerlist.append(innerlist)